In [3]:
%pip install vocode

  Using cached vocode-0.1.111-py3-none-any.whl (4.7 MB)
  Using cached anthropic-0.2.10-py3-none-any.whl (6.3 kB)
  Using cached websockets-11.0.3-cp39-cp39-macosx_11_0_arm64.whl (121 kB)
  Using cached fastapi-0.100.1-py3-none-any.whl (65 kB)
  Using cached python_multipart-0.0.6-py3-none-any.whl (45 kB)
  Using cached langchain-0.0.198-py3-none-any.whl (1.0 MB)
  Using cached SpeechRecognition-3.10.4-py2.py3-none-any.whl (32.8 MB)
  Using cached openai-0.27.10-py3-none-any.whl (76 kB)
  Using cached uvicorn-0.21.1-py3-none-any.whl (57 kB)
  Using cached scipy-1.13.1-cp39-cp39-macosx_12_0_arm64.whl (30.3 MB)
  Using cached starlette-0.27.0-py3-none-any.whl (66 kB)
  Using cached numexpr-2.10.1-cp39-cp39-macosx_11_0_arm64.whl (130 kB)
  Using cached openapi_schema_pydantic-1.2.4-py3-none-any.whl (90 kB)
     |████████████████████████████████| 2.1 MB 1.6 MB/s eta 0:00:01
  Using cached pydantic-1.10.17-cp39-cp39-macosx_11_0_arm64.whl (2.3 MB)
  Using cached langchainplus_sdk-0.0.20-py3-

In [2]:
import asyncio
import signal
import uuid

from pydantic_settings import BaseSettings, SettingsConfigDict

from vocode.helpers import create_streaming_microphone_input_and_speaker_output
from vocode.streaming.agent.chat_gpt_agent import ChatGPTAgent
from vocode.streaming.models.agent import ChatGPTAgentConfig
from vocode.streaming.models.message import BaseMessage
from vocode.streaming.models.synthesizer import AzureSynthesizerConfig
from vocode.streaming.models.transcriber import (
    DeepgramTranscriberConfig,
    PunctuationEndpointingConfig,
)
from vocode.streaming.streaming_conversation import StreamingConversation
from vocode.streaming.synthesizer.azure_synthesizer import AzureSynthesizer
from vocode.streaming.transcriber.deepgram_transcriber import DeepgramTranscriber
from flowise_agent import FlowiseAgent, FlowiseAgentConfig

# configure_pretty_logging()


class Settings(BaseSettings):
    """
    Settings for the streaming conversation quickstart.
    These parameters can be configured with environment variables.
    """

    openai_api_key: str = "sk-proj-8dMh435vAkkx70BD3zP1T3BlbkFJJSkuEushVxuEQ5PYAkkw"
    azure_speech_key: str = "d31436658f234e619bab457ccbb8bcf1"
    deepgram_api_key: str = "92c1f9b1e9e58fadd18cd5d646346499b59d8959"

    azure_speech_region: str = "eastus"

    # This means a .env file can be used to overload these settings
    # ex: "OPENAI_API_KEY=my_key" will set openai_api_key over the default above
    model_config = SettingsConfigDict(
        env_file=".env",
        env_file_encoding="utf-8",
        extra="ignore",
    )


settings = Settings()


async def main():
    (
        microphone_input,
        speaker_output,
    ) = create_streaming_microphone_input_and_speaker_output(
        use_default_devices=True,
    )
    synconfig = AzureSynthesizerConfig.from_output_device(speaker_output)
    synconfig.voice_name = "en-GB-RyanNeural"
    synconfig.language_code = "en-GB"
    conversation = StreamingConversation(
        output_device=speaker_output,
        transcriber=DeepgramTranscriber(
            DeepgramTranscriberConfig.from_input_device(
                microphone_input,
                endpointing_config=PunctuationEndpointingConfig(),
                api_key=settings.deepgram_api_key,
            ),
        ),
        agent=FlowiseAgent(  # Use FlowiseAgent
            FlowiseAgentConfig(
                flowise_api_url="https://dev-llm.deeptalk.work/api/v1/prediction/c7686ede-9366-4d66-967c-7b56a790afe4",
                chat_id=str(uuid.uuid4())
            )
        ),
        synthesizer=AzureSynthesizer(
            synconfig,
            azure_speech_key=settings.azure_speech_key,
            azure_speech_region=settings.azure_speech_region,
        ),
    )
    await conversation.start()
    print("Conversation started, press Ctrl+C to end")
    signal.signal(signal.SIGINT, lambda _0, _1: asyncio.create_task(conversation.terminate()))
    while conversation.is_active():
        chunk = await microphone_input.get_audio()
        conversation.receive_audio(chunk)


if __name__ == "__main__":
    await main()

Conversation started, press Ctrl+C to end
 Nine one one. What is your emergency?
There's a fire in my apartment building! I'm in the third floor, apartment 3B. The address is 125 Maple Street, Rivertown. The smoke is coming from the floor below, and it's getting hard to breathe. Please send help quickly!
 Okay. We are sending the fire and rescue team.
Thank you so much. I'm trying to stay low to avoid the smoke. There are other people in the building too; I can hear them yelling for help. Please hurry!
 Okay. Approximately, how many people will be there in the building.
There are about ten apartments per floor, and it's a three-story building. I would estimate around 25 to 30 people could be in the building, but I can't be sure.
 Has anyone been injured by smoke or fire?
I can't see anyone else from where I am, but I can hear coughing and yelling, so it's possible that some people are affected by the smoke. I'm starting to feel dizzy myself.
 Okay.
 Is there any windows nearby you can 

: 